# Neuron set examples

In [1]:
import os

import obi_one as obi

### __Initialization:__ Loading a circuit

In [2]:
circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/"
circ_path = circuit_path_prefix + "O1_data/circuit_config.json"
circuit = obi.Circuit(name="SSCX_O1", path=circ_path)
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")

Circuit 'SSCX_O1' with 785632 neurons and 560631825 synapses
Default node population: 'S1nonbarrel_neurons'


### __Example 1:__ Adding node set dict to an existing SONATA circuit object + writing new node set .json file

In [3]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Adding multiple node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
print("..." + str(c.node_sets.content)[-124:])

# Add node set from NeuronSet object, resolved in circuit's default node population
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
print("..." + str(c.node_sets.content)[-168:])

# Adding a node sets based on previously added node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
print("..." + str(c.node_sets.content)[-216:])

# Write new circuit's node set file
obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

...211709], 'layer': ['6']}}
...211709], 'layer': ['6']}, 'Layer23': {'layer': [2, 3]}}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3'], 'AllLayers': ['Layer123', 'Layer4', 'Layer56']}


### __Example 2:__ Use of different NeuronSet types

<u>Important</u>: In general, the validity of neuron set definitions is not checked during initialization, but when resolved within a specific circuit's node population

#### (a) __ExistingNeuronSet__, wrapper for an existing node set
<u>Note:</u> This neuron set does not resolve to a dict, since the underlying node set is already existing by definition

In [4]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=None)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (2695): [    0     1     2 ... 35663 35664 35665]
> Node set dict: ['Layer1']


#### (b) __ExistingNeuronSet__, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

<u>Note 2</u>: Random sub-sampling will enforce resolving into a new node set

In [5]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", random_sample=10, random_seed=1)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (10): [   41    54 33735 33794 33870 34176 34777 34870 34916 35004]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [41, 54, 33735, 33794, 33870, 34176, 34777, 34870, 34916, 35004]}


#### (c) __CombinedNeuronSet__, based on combining existing (named) node sets

In [6]:
neuron_set = obi.CombinedNeuronSet(circuit=circuit, population="All", node_sets=("Layer1", "Layer2", "Layer3"), random_sample=None)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (63530): [     0      1      2 ... 170851 170852 170853]
> Node set dict: ['Layer1', 'Layer2', 'Layer3']


#### (d) __CombinedNeuronSet__, based on combining existing (named) node sets, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [7]:
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"), random_sample=10, random_seed=0)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (10): [  1118  13098  13437  18214  30537  31342  34865 146702 151954 155457]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1118, 13098, 13437, 18214, 30537, 31342, 34865, 146702, 151954, 155457]}


#### (e) __IDNeuronSet__, based on individual neuron IDs

In [8]:
neuron_set = obi.IDNeuronSet(neuron_ids=(1, 2, 3))
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (3): [1 2 3]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 2, 3]}


#### (f) __IDNeuronSet__, based on individual neuron IDs, with random sub-sampling
<u>Note</u>: `random_sample` can be an absolute number or fraction

In [9]:
neuron_set = obi.IDNeuronSet(neuron_ids=range(10), random_sample=0.5, random_seed=999)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (5): [1 3 4 7 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 3, 4, 7, 9]}


#### (g) __PropertyNeuronSet__, based on neuron properties
<u>Note</u>: Optionally, instead of keeping the synbolic notation, neuron IDs can be resolved to individual IDs by `force_resolve_ids=True`.

In [10]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

# # Optional: Individual neuron IDs resolved
print(f"> Node set dict with IDs resolved [OPTIONAL]: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name, force_resolve_ids=True)}")

AttributeError: 'NeuronPropertyFilter' object has no attribute 'enforce_no_lists'

#### (h) __PropertyNeuronSet__, based on neuron properties, combined with exising (named) node sets
<u>Note</u>: In this case, individual neuron IDs will always be resolved since a combination of properties and node sets is not possible in SONATA node sets otherwise!

In [ ]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

#### (i) __VolumetricCountNeuronSet__, sample a spatial neighborhood


In [ ]:
neuron_set = obi.VolumetricCountNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    n=10,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

#### (j) __VolmetricRadiusNeuronSet__, also neigborhood, but fixed radius
<u>Note</u>: Can be combined with randm subsampling

In [ ]:
neuron_set = obi.VolumetricRadiusNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    radius=75.0,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
    random_sample=5
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

### __Example 3:__ Writing a NeuronSet to a SONATA node set file
<u>Note</u>: A NeuronSet name must be set, which will be the name of the SONATA node set. The name must not exist!

<u>Note 2</u>: The node sets file name is by default taken from the original circuit. An alternative name can optionally be provided.

<u>Note 3</u>: Overwrite (`overwrite_if_exists`) and append (`append_if_exists`) options exist.

In [ ]:
output_path = "./"

# Write new file, overwrite if existing
neuron_set = obi.CombinedNeuronSet(name="L123", node_sets=("Layer1", "Layer2", "Layer3"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, overwrite_if_exists=True)

# Append to existing file, but name already exists => NOT POSSIBLE
# nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path="./", append_if_exists=True)  # AssertionError: Appending not possible, node set 'Basic' already exists!

# Append to existing file
neuron_set = obi.CombinedNeuronSet(name="L456", node_sets=("Layer4", "Layer5", "Layer6"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, append_if_exists=True)

if os.path.exists(nset_file):
    print(f"Node set file: {nset_file}")